<a href="https://colab.research.google.com/github/daichi-a/PythonDataAnalyzeClassMaterial/blob/master/PyDataAnalysis_ANOVA_Tukey_ID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSojvSyQmboSLQTHF__Fm4MgQrnXOoRHpO4PKR9jKUlkU1KIZtL6k5OzYjbiGVRUlrfBWDXL6xkYwy_/pub?gid=655120483&single=true&output=csv')
df.head()

,平均気温,群
0,0.7,青梅
1,2.6,青梅
2,3.7,青梅
3,2.6,青梅
4,1.1,青梅


In [2]:
print(df)

     平均気温   群
0     0.7  青梅
1     2.6  青梅
2     3.7  青梅
3     2.6  青梅
4     1.1  青梅
..    ...  ..
121   4.3  府中
122   4.6  府中
123   4.8  府中
124   4.8  府中
125   3.5  府中

[126 rows x 2 columns]


In [8]:
# Bartlett検定
import scipy.stats

# scipyのbartlett検定は，t検定の時のように，群ごとのnumpy配列を取る．
# つまり，42個ずつの「平均気温」だけの値が入ったnumpy配列を作成すれば良い
# pandas.DataFrameは取れないことに注意
# （アンスタックデータでは列だけ取り出せば良いので簡単だが，今回はスタック形式にしたので，ちょっと操作をする）

ome_ave = df.loc[0:41, '平均気温'].values # 平均気温列の0行目〜41行目を取り出すと，そのオブジェクトの中のvalues変数にはnumpy配列が入っている
hachioji_ave = df.loc[42:83, '平均気温'].values
fuchu_ave = df.loc[84:, '平均気温'].values

bartlett_result = scipy.stats.bartlett(ome_ave, hachioji_ave, fuchu_ave)
print(bartlett_result)


BartlettResult(statistic=0.009294251494411373, pvalue=0.9953636554346814)


In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# formula(式)を与えて線形回帰を呼び出す
ols_lm = smf.ols(formula='平均気温 ~ 群', data=df).fit()
# 線形回帰の結果をもとにType2の分散分析をかける
anova_result = sm.stats.anova_lm(ols_lm, typ=2)

print(anova_result)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


              sum_sq     df          F        PR(>F)
群          39.003333    2.0  15.758491  8.074710e-07
Residual  152.216667  123.0        NaN           NaN


In [13]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey_result_jan = pairwise_tukeyhsd(df.loc[:, '平均気温'].values, df.loc[:, '群'].values)
print(tukey_result_jan)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
   八王子     府中   1.0095 0.001  0.4336  1.5855   True
   八王子     青梅  -0.2881 0.465 -0.8641  0.2879  False
    府中     青梅  -1.2976 0.001 -1.8736 -0.7217   True
---------------------------------------------------
